In [1]:
'''
Code snippet showing the implementation of the reference epoch algorithm
described in this thesis. An important function for the frame transformation
is described first.
'''


import numpy as np
import gwsurrogate as gws
import scri



def from_iner_to_copr(t, h, ell_min, ell_max):
    '''
    Transforms modes from the inertial frame into the coprecessing frame.
    
    Parameters
    ----------
    t : ndarray
        Time array.
    h : dict
        Dictionary of available modes with (ell, m) tuples as keys in
        the inertial frame.
    ell_min : int
        Minimum ell value.
    ell_max : int
        Maximum ell value.
    
    Returns
    -------
    dict
        Dictionary of modes in the coprecessing frame.
    '''
    # Build scri WaveformModes object
    data = list(h.values())
    data = np.array(data).T
    waveform_modes = scri.WaveformModes(
                            dataType=scri.h,
                            t=t,
                            data=data,
                            ell_min=ell_min,
                            ell_max=ell_max,
                            frameType=scri.Inertial,
                            r_is_scaled_out=True,
                            m_is_scaled_out=True)

    # Apply scri frame transformation
    waveform_modes.to_coprecessing_frame()
    data_copr = waveform_modes.data.T
    
    # Build mode dictionary 
    mode_list = [(ell, m) for ell in range(ell_min, ell_max+1) 
                 for m in range(-ell, ell+1)]
    h_copr = dict(zip(mode_list, data_copr))
    
    return h_copr



def get_f_ref(q, chi1, chi2, t_ref):
    '''
    Transforms a reference time in units of M to the referencefrequency in
    units of cycles/M, which is needed to set the reference epoch in NRSur7dq4.
    
    Parameters
    ----------
    q : float
        Mass ratio m1/m2 >= 1.
    chi1 : ndarray
        Dimensionless spin of the heavier black hole.
    chi2 : ndarray
        Dimensionless spin of the lighter black hole.
    t_ref : float
        Reference time between -4300 M and -100 M to define spins.
    
    Returns
    -------
    float
        Reference frequency in cycles/M.
    '''
    # Generate surrogate waveform GW^0 at t^0_{ref} = -4300 M
    sur = gws.LoadSurrogate('NRSur7dq4') 
    dt = 0.1
    ell_min = 2
    ell_max = 4
    t, h, dyn = sur(q, chi1, chi2, dt=dt, f_low=0) 
    h_copr = from_iner_to_copr(t, h, ell_min, ell_max)
    
    # Get frequency-time map from GW phase
    phi_GW0 = 1/2 * (np.angle(h_copr[(2, -2)]) - np.angle(h_copr[(2, 2)])) 
    phi_GW0_dot = np.diff(phi_GW0) / dt                                                            
    phi_GW0_dot = np.append(phi_GW0_dot[0], phi_GW0_dot)                                                                                                          
    
    # Substitute t_{ref} and multiply conversion factor
    i_ref = np.argmin(np.abs(t - t_ref))
    f_ref = phi_GW0_dot[i_ref] / (2*np.pi)  
    
    return f_ref



# Example output
q = 2
chi1 = [.5, .2, .5]
chi2 = [-.3, .1, .5]
t_ref = -500
f_ref = get_f_ref(q, chi1, chi2, t_ref)
print('The GW frequency at t = %.1f M is f = %.4f cyc/M' % (t_ref, f_ref))

lal.MSUN_SI != Msun
setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new
setting __package__ to gwsurrogate.new so relative imports work
setting __package__ to gwsurrogate.new so relative imports work
Loaded NRSur7dq4 model
The GW frequency at t = -500.0 M is f = 0.0124 cyc/M
